In [ ]:
from ase.neb import NEB,NEBOptimizer, NEBTools
from ase.optimize import MDMin, FIRE, BFGS, LBFGS, GPMin, QuasiNewton
import ase
from ase import Atoms
from ase.io import write, read
from ase.visualize import view
from openbabel import openbabel as ob
from ase.calculators.dftb import Dftb
from spooks.spookynet import SpookyNetCalculator, spookynet
from matplotlib import pyplot as plt
from ase.constraints import FixAtoms

def add_h(atoms):
    ase.io.write('./temph.xyz', atoms)
    obConversion = ob.OBConversion()
    obConversion.SetInAndOutFormats("xyz", "mol2")
    mol = ob.OBMol()
    obConversion.ReadFile(mol, "temph.xyz")  
    obConversion.WriteFile(mol, 'temph.mol2')
    obConversion = ob.OBConversion()
    obConversion.SetInAndOutFormats("mol2", "xyz")
    mol = ob.OBMol()
    obConversion.ReadFile(mol, "temph.mol2")  
    mol.ConnectTheDots()
    mol.PerceiveBondOrders()
    mol.AddHydrogens()
    obConversion.WriteFile(mol, './temph.xyz')
    atoms = ase.io.read('./temph.xyz')
    return atoms

molecule_num = 40849 #39583, 40004

path = './plottini/neb_data/molecule_{}/'.format(molecule_num)
initial = read(path+'conf_0H.xyz')
#initial = add_h(initial)
initial.set_calculator(SpookyNetCalculator(load_from="./spooks/qm7x_known_unknown_0.pth", charge=0, magmom=0))
opt = BFGS(initial)
opt.run(fmax = 0.0011, steps = 200)

final = read(path+'conf_6H.xyz')
#final = add_h(final)
final.set_calculator(SpookyNetCalculator(load_from="./spooks/qm7x_known_unknown_0.pth", charge=0, magmom=0))
opt = BFGS(final)
opt.run(fmax = 0.0011, steps = 200)

images = [initial]
images += [(read(path + 'conf_{}H.xyz'.format(i))) for i in range(1,6)]
images += [final]

import rmsd
pos_old = images[0].get_positions()

for i in range(1,7):
    
    image = images[i]
    pos = image.get_positions()
    pos = pos - rmsd.centroid(pos)
    rotation = rmsd.quaternion_rotate(pos, pos_old)
    pos = pos@rotation
    images[i].set_positions(pos)

neb = NEB(images, remove_rotation_and_translation=True)
images[0].set_calculator()
images[-1].set_calculator()
constraint = FixAtoms(mask=[atom.symbol != 'Cu' for atom in initial])
calcs = []
for i in range(0,6):
    
    calc = SpookyNetCalculator(load_from="./spooks/qm7x_known_unknown_0.pth", charge=0, magmom=0)
    calcs.append(calc)
    images[i].calc = calcs[-1]
    
for i in [0,-1]:
    constraint = FixAtoms(mask=[atom for atom in images[i]])

optimizer = NEBOptimizer(neb)
optimizer.run()

optimizer = NEBOptimizer(neb)
optimizer.run()

energy = []
images[0].set_calculator(calc)
images[-1].set_calculator(calc)
for image in images:
    energy.append(image.get_total_energy())
plt.plot(energy)

In [ ]:
import pandas as pd
import numpy as np
dataplot = pd.read_json('./plottini/neb_data/dati_plottino_{}.json'.format(molecule_num))
fig, ax = plt.subplots(1,1, figsize = (5,4), dpi = 800)
img = ax.plot(np.arange(0,7), dataplot-np.sort(dataplot.values[0]), color='navy', marker='.', linestyle='dashed', linewidth=1.5,  mfc='none', mec='navy',mew='2', markersize=16, label = 'geodesic interpolation')
ax.plot(np.arange(0,7), np.array(energy)-np.array(energy).min(), color='crimson', marker='x', linestyle='dashed', linewidth=1.5, markersize=10, label = 'SpookyNet NEB opt')

plt.tick_params(left='on')
plt.ylabel(r'$\Delta E \;\; \mathrm{(eV)}$', fontsize=14)
plt.xlabel('interpolation step', fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()